In [ ]:
# data_loader

https://pnnl-compbio.github.io/coderdata/pages/usage

In [ ]:
cd data/

In [ ]:
pwd
# mdl-sys-bnchmrk/data

In [ ]:
import coderdata as cd

In [ ]:
# cd.download_data_by_prefix('beataml')
cd.download_data_by_prefix('cell_line')
cd.download_data_by_prefix('cptac')
# cd.download_data_by_prefix('hcmi')

In [ ]:
cptac = cd.DatasetLoader('cptac') # a
cell_line = cd.DatasetLoader('cell_line') # b

print('DatasetLoaders in RAM')
projects = 'cell_line_CPTAC'

In [ ]:
import glob
import pandas as pd
import umap
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [ ]:
ls ..

In [ ]:
ls ../output/

In [ ]:
out = '../output/dbl_lbl/'

In [ ]:
u_o = '../output/umap/'

In [ ]:
# Traspose long-form X-file into biomarkers as columns,
def extract(df_lite):
    import time
    start = time.time()
    data_type = df_lite.columns[2] # extract marker values

    dot_T = df_lite.pivot_table(
        index='improve_sample_id',
        columns='entrez_id',
        values=data_type,
        aggfunc='mean'             # average duplicate values
    )

    end = time.time()
    wall_clock = end - start
    return str(round(wall_clock / 60, 2)) + ' minutes', dot_T

In [ ]:
# Extract ids and biomarker values
def df_check(X_n):
    df_lite = X_n.iloc[:, :3] # cut the last two columns, source and study
    size = f"{df_lite.shape[0]:,}"
    na_count = f"{df_lite.isna().sum().sum():,}"
    inf_count = f"{df_lite.isin([np.inf, -np.inf]).sum().sum():,}"
    return df_lite, size, na_count, inf_count

In [ ]:
# dot_T = g(d_typ, dot_T.copy())
def g(d_typ, df):
    """
    Checks the data types of columns and index in a DataFrame and prints informative messages.

    Args:
        df (pandas.DataFrame): The DataFrame to check.

    Returns:
        None
    """

    if df.columns.dtype == 'float64' and df.index.dtype == 'float64':
        print('both float')
        df = float_to_string(d_typ, df)
    elif df.columns.dtype == 'float64' and df.index.dtype == 'int':
        print('columns are float, index are int')
        df = indx_int_colm_flt(d_typ, df)
    elif df.columns.dtype == 'int' and df.index.dtype == 'float':
        print('columns are int, index are float, fail, write another function')
        # forth function
    elif df.columns.dtype == 'int' and df.index.dtype == 'int':
        print('columns are int, index are int')
        df = int_to_string(d_typ, df)
    else:
        print('non int or float dtype detected')
    return df

In [ ]:
def int_to_string(d_typ, dot_T):
    dot_T.columns = dot_T.columns.map(str)
    dot_T.columns = ['entrz_' + d_typ + i for i in dot_T.columns] #
    dot_T.columns.name = 'entrez_id'

    dot_T.index = dot_T.index.map(str)
    dot_T.index = ['smpl_id_' + i for i in dot_T.index]
    dot_T.index.name = 'improve_sample_id'
    return dot_T

In [ ]:
def indx_int_colm_flt(d_typ, dot_T):
    dot_T.columns = dot_T.columns.map(str)
    dot_T.columns = [i.split('.')[0] for i in dot_T.columns]
    dot_T.columns = ['entrz_' + d_typ + i for i in dot_T.columns]
    dot_T.columns.name = 'entrez_id'
    
    dot_T.index = dot_T.index.map(str)
    dot_T.index = ['smpl_id_' + i for i in dot_T.index]
    dot_T.index.name = 'improve_sample_id'
    return dot_T

In [ ]:
def float_to_string(d_typ, dot_T):
    dot_T.columns = dot_T.columns.map(str)
    dot_T.columns = [i.split('.')[0] for i in dot_T.columns]
    dot_T.columns = ['entrz_' + d_typ + i for i in dot_T.columns]
    dot_T.columns.name = 'entrez_id'
    
    dot_T.index = dot_T.index.map(str)
    dot_T.index = [i.split('.')[0] for i in dot_T.index]
    dot_T.index = ['smpl_id_' + i for i in dot_T.index]
    dot_T.index.name = 'improve_sample_id'
    return dot_T

#### Samples, abstract systems to a and b

In [ ]:
proj_a_samp = cell_line.samples
sys_a = 'cell_line'

proj_b_samp = cptac.samples
sys_b = 'cptac'

#### Cancer_type toggles, cell line as a, CPTAC as b

In [ ]:
# Lung adeno
cancer = 'lung_adeno'
proj_a_samp_canc_n = proj_a_samp[proj_a_samp.cancer_type == 'Lung Adenocarcinoma'] # cl
proj_b_samp_canc_n = proj_b_samp[proj_b_samp.cancer_type == 'Lung adenocarcinoma']

In [ ]:
# Pancreatic adeno
cancer = 'pancreatic_adeno'
proj_a_samp_canc_n = proj_a_samp[proj_a_samp.cancer_type == 'Pancreatic Adenocarcinoma'] # cl, get cheks v4
proj_b_samp_canc_n = proj_b_samp[proj_b_samp.cancer_type == 'Pancreatic ductal adenocarcinoma']

In [ ]:
# Head and neck
cancer = 'head_neck'
proj_a_samp_canc_n = proj_a_samp[proj_a_samp.cancer_type == 'Head and Neck squamous cell carcinoma']
proj_b_samp_canc_n = proj_b_samp[proj_b_samp.cancer_type == 'Head and Neck Squamous Cell Carcinoma']

In [ ]:
# Colon adeno
cancer = 'colon_adeno'
proj_a_samp_canc_n = proj_a_samp[proj_a_samp.cancer_type == 'Colon adenocarcinoma']
proj_b_samp_canc_n = proj_b_samp[proj_b_samp.cancer_type == 'Colon Adenocarcinoma']

In [ ]:
# Glioblastoma
cancer = 'glioblastoma'
proj_a_samp_canc_n = proj_a_samp[proj_a_samp.cancer_type == 'Glioblastoma multiforme']
proj_b_samp_canc_n = proj_b_samp[proj_b_samp.cancer_type == 'Glioblastoma']

In [ ]:
# Breast Ductal
cancer = 'breast_ductal'
proj_a_samp_canc_n = proj_a_samp[proj_a_samp.cancer_type == 'Breast carcinoma']
proj_b_samp_canc_n = proj_b_samp[proj_b_samp.cancer_type == 'Breast Invasive Ductal Carcinoma']

In [ ]:
# Breast Lobular
cancer = 'breast_lobular'
proj_a_samp_canc_n = proj_a_samp[proj_a_samp.cancer_type == 'Breast carcinoma']
proj_b_samp_canc_n = proj_b_samp[proj_b_samp.cancer_type == 'Breast Invasive Lobular Carcinoma']

In [ ]:
# Breast NOS
cancer = 'breast_nos'
proj_a_samp_canc_n = proj_a_samp[proj_a_samp.cancer_type == 'Breast carcinoma']
proj_b_samp_canc_n = proj_b_samp[proj_b_samp.cancer_type == 'Breast Invasive Carcinoma, NOS']

In [ ]:
# Renal clear cell
cancer = 'renal_clear_cell'
proj_a_samp_canc_n = proj_a_samp[proj_a_samp.cancer_type == 'Clear cell renal cell carcinoma']
proj_b_samp_canc_n = proj_b_samp[proj_b_samp.cancer_type == 'Renal Clear Cell Carcinoma']

#### Continue

In [ ]:
ids_proj_a = proj_a_samp_canc_n.improve_sample_id # cl
ids_proj_b = proj_b_samp_canc_n.improve_sample_id # cp

#### Data extraction

In [ ]:
data_type = 'transcriptomics' # to file name
d_typ = 'tran_' # to columns and index
dta_n_proj_a = cell_line.transcriptomics[cell_line.transcriptomics.improve_sample_id.isin(ids_proj_a)] # cl
dta_n_proj_b = cptac.transcriptomics[cptac.transcriptomics.improve_sample_id.isin(ids_proj_b)]

In [ ]:
data_type = 'proteomics' # to file name
d_typ = 'prot_' # to columns and index
dta_n_proj_a = cell_line.proteomics[cell_line.proteomics.improve_sample_id.isin(ids_proj_a)] # cl
dta_n_proj_b = cptac.proteomics[cptac.proteomics.improve_sample_id.isin(ids_proj_b)]

In [ ]:


dta_n_proj_a = cell_line.copy_number[cell_line.copy_number.improve_sample_id.isin(ids_proj_a)]
dta_n_proj_b = cell_line.copy_number[cell_line.copy_number.improve_sample_id.isin(ids_proj_b)]

In [ ]:


dta_n_proj_a = cptac.mutations[cptac.mutations.improve_sample_id.isin(ids_proj_a)]
dta_n_proj_b = cell_line.mutations[cell_line.mutations.improve_sample_id.isin(ids_proj_b)]

#### Project A, cell line

In [ ]:
df_lite, size, na_count, inf_count = df_check(dta_n_proj_a)
print(sys_a) # sa
print(cancer, data_type)
print(size)
print(na_count)
print(inf_count)

In [ ]:
wall_clock, dot_T = extract(df_lite)
print(wall_clock)
dot_T.head(1).iloc[:, :1]

In [ ]:
d_typ

In [ ]:
dot_T = g(d_typ, dot_T.copy())
dot_T.head(1).iloc[:, :1]

In [ ]:
dot_T.shape

In [ ]:
dot_T.dropna(axis = 1, inplace = True)

In [ ]:
dot_T.shape

In [ ]:
a = dot_T # cell line

#### Project B, CPTAC

In [ ]:
df_lite, size, na_count, inf_count = df_check(dta_n_proj_b)
print(sys_b)
print(cancer, data_type)
print(size)
print(na_count)
print(inf_count)

In [ ]:
wall_clock, dot_T = extract(df_lite)
print(wall_clock)
dot_T.head(1).iloc[:, :1]

In [ ]:
dot_T = g(d_typ, dot_T.copy())
# dot_T.head(1).iloc[:, :1]

In [ ]:
dot_T.head(1).iloc[:, :1]

In [ ]:
dot_T.shape

In [ ]:
dot_T.dropna(axis = 1, inplace = True)

In [ ]:
dot_T.shape

In [ ]:
b = dot_T # cptac

#### Labels

In [ ]:
a.insert(0, 'Labels', 'cell_line')
b.insert(0, 'Labels', 'CPTAC')

In [ ]:
ab = pd.concat([a, b], axis=0, join='inner')

In [ ]:
ab

In [ ]:
out, u_o, cancer, data_type

In [ ]:
# want to write double label - idea run double label on mbatch
# do we want single label also as benchmarks? - compare mbatch and vae on sing lab?
ab.to_csv(
    '../output/sing_lab/'+cancer+'_'+data_type+'_'+projects+'.tsv',
    sep = '\t')

In [ ]:
check = pd.read_csv('../output/sing_lab/'+cancer+'_'+data_type+'_'+projects+'.tsv',
                    sep = '\t', index_col = 0)

In [ ]:
print(cancer)
check

#### Dbl Lbl

In [ ]:
ab = ab.rename(columns={'Labels': 'System'})

In [ ]:
ab.head(1).iloc[:, :1]

In [ ]:
ab.insert(1, 'Cancer_type', cancer)

In [ ]:
ab.iloc[:3, :3]

In [ ]:
ab.shape

In [ ]:
check.iloc[:3, :3]

In [ ]:
check.shape

In [ ]:
check.insert(1, 'Cancer_type', 'lung_adeno')

In [ ]:
check.iloc[:3, :3]

In [ ]:
check = check.rename(columns={'Labels': 'System'})

In [ ]:
dbl_lbl = pd.concat([ab, check], axis = 0, join = 'inner')

In [ ]:
dbl_lbl

In [ ]:
need c1, c2 vars

In [ ]:
cancer

In [ ]:
dbl_lbl.to_csv('../output/dbl_lbl/lung_adeno+'+cancer+'_'+data_type+'_'+projects+'.tsv',
                    sep = '\t')

In [ ]:
dbl_lbl

#### UMAP

In [ ]:
# Lead-in for UMAP - read from sing_lab
    # really want to test adding synth samples - single cell

In [ ]:
ab.insert(0, 'Data_phase', 'Original')

In [ ]:
ab